# PyOTA "Hello world" tutorial.

The goal of this tutorial is to create new empty wallet based on new random seed and to transfer IOTA tokens to it from wallet generated via faucet.

## Imports
Currently we have following imports:
 * iota - module name for [PyOTA](https://pyota.readthedocs.io/en/latest/)
 * secrets - "The [secrets](https://docs.python.org/3/library/secrets.html#module-secrets) module is used for generating cryptographically strong random numbers suitable for managing data such as passwords, account authentication, security tokens, and related secrets."
 * requests - "[Requests](http://docs.python-requests.org/en/master/) is an elegant and simple HTTP library for Python, built for human beings."

In [1]:
import iota
import secrets
import requests

## Generating and printing receiver seed
We can easily and securely generate seed which will be used as seed for receiver side.

In [2]:
chars = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ9'
receiver_seed = ''
for i in range(81): receiver_seed += secrets.choice(chars)
    
print(receiver_seed)

FFPCNTKEMMJUHUBFUYVVMQAEGFJFDABBXR9UVBMXSEIPZQQACBYNDJKKGSCAJFPZMOIVQJKUWFDT9ZKUH


## Connecting to testnet
Since we would like to be as close to actual Main IOTA network we will use IOTA testnet. IOTA testnet is supposed to be almost identical in operating as mainnet but has faster confirmation times and is reseted regurally, so tokens used there have no value. Here is list of example testnet nodes:

* http://p103.iotaledger.net:14700
* https://testnet140.tangle.works:443
* http://p101.iotaledger.net:14700

Please note that they could be down or unresponsive. 

In [3]:
api = iota.Iota('https://testnet140.tangle.works:443', seed=receiver_seed)
api.get_node_info()

{'appName': 'IRI Testnet',
 'appVersion': '1.4.2.1',
 'duration': 0,
 'jreAvailableProcessors': 8,
 'jreFreeMemory': 218798104,
 'jreMaxMemory': 15271460864,
 'jreTotalMemory': 1064828928,
 'jreVersion': '1.8.0_161',
 'latestMilestone': TransactionHash(b'HGKJHP9WLIVHWWECHVAEMOFZQQP9DJHEBKGYXIGHYUSJRZ9MST9SEQPL9HJMAPNUGQLY9NEDWTDFGL999'),
 'latestMilestoneIndex': 318430,
 'latestSolidSubtangleMilestone': TransactionHash(b'999999999999999999999999999999999999999999999999999999999999999999999999999999999'),
 'latestSolidSubtangleMilestoneIndex': 1000,
 'neighbors': 4,
 'packetsQueueSize': 0,
 'time': 1518704333104,
 'tips': 0,
 'transactionsToRequest': 2}

## Prepare receiver address
We already have seed for receiver (in variable **receiver_seed**), now we would like to have his address. Lets generate one:

In [4]:
gna_result = api.get_new_addresses(count=1)
addresses = gna_result['addresses']
receiver_address = addresses[0]
print("Receiver address 0 is: " + str(receiver_address))

Receiver address 0 is: BIRMZLDANYVITFMNEZONMGDOVZACPP9BHJMHPWMGAGGBUFI9SWTKQYNBZI9EDVSXQCDBFRVURV9YNDGVX


## Prepare sender - how to get testnet tokens?
If we are generating new wallets from new seeds then thoose wallets will be empty (well, [probably](https://matthewwinstonjohnson.gitbooks.io/iota-guide-and-faq/how-secure-is-my-seed.html)). Since we would like to simulate transfer of "value" between two parties, we would need to have some tokens in at least one wallet in our setup. Thanks to IOTA devs, there is service called faucet which is genearting seeds and wallet addreses with some testnet tokens in it (2K at the moment). They also provide JSON API so we can use it right away.

In [5]:
r = requests.get('https://seeedy.tangle.works/')
print("Faucet response: " + str(r.status_code))
sender_wallet = r.json()

Faucet response: 200


In [6]:
print("Sender seed: " + sender_wallet["seed"])
print("Sender address: " + sender_wallet["address"])
print("Sender amount: " + str(sender_wallet["amount"]))

Sender seed: AXTI9CAUCGMQYCEJEZQNVPTZTPDAHGIPNTWFVNQGXZSYGO9ISRPJVTQZKIBHBCTM9WJHWGADM9RBYKSIE
Sender address: RXZHKUFSGRMKJHCDVMVSREBLROC9AL9NXQONOHCRWYFTOXCDWTPDJIGJQAFEDEJGNENP9QESIYIFAONVX
Sender amount: 2000


## Prepare transaction

In [7]:
message = ("Here, have all my testnet tokens!")

In [8]:
proposedTransaction = iota.ProposedTransaction(address = iota.Address(receiver_address), 
                                               value = sender_wallet["amount"], 
                                               message = iota.TryteString.from_string(message)
                                              )

In [11]:
proposedTransaction

ProposedTransaction(**{'address': Address(b'BIRMZLDANYVITFMNEZONMGDOVZACPP9BHJMHPWMGAGGBUFI9SWTKQYNBZI9EDVSXQCDBFRVURV9YNDGVX'),
                     'attachment_timestamp': 0,
                     'attachment_timestamp_lower_bound': 0,
                     'attachment_timestamp_upper_bound': 0,
                     'branch_transaction_hash': TransactionHash(b'999999999999999999999999999999999999999999999999999999999999999999999999999999999'),
                     'bundle_hash': None,
                     'current_index': None,
                     'hash_': None,
                     'last_index': None,
                     'legacy_tag': Tag(b'999999999999999999999999999'),
                     'nonce': Nonce(b'999999999999999999999999999'),
                     'signature_message_fragment': None,
                     'tag': Tag(b'999999999999999999999999999'),
                     'timestamp': 1518704335,
                     'trunk_transaction_hash': TransactionHash(b'999999999999999

In [9]:
api = iota.Iota('https://testnet140.tangle.works:443', seed=sender_wallet["seed"])

In [10]:
api.send_transfer(transfers = [proposedTransaction], 
                  depth = 1,
                  inputs = [iota.Address(sender_wallet["address"], key_index=0, security_level=2)]
                 )

BadApiResponse: 400 response from node: Tip not found: 999999999999999999999999999999999999999999999999999999999999999999999999999999999

## Versions of modules used in tutorial:

In [ ]:
assert iota.__version__ == '2.0.4'
assert requests.__version__ == '2.18.4'